In [ ]:
!pip install spacy

In [ ]:
!pip install PyMuPDF

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
!pip install gradio

In [ ]:
import fitz # PyMuPDF
import spacy
import gradio as gr
import re

print("Environment Ready!")

In [9]:
def extract_text_from_pdf(pdf_file):
  """
  PDF text reader
  """

  try:
    doc = fitz.open(pdf_file.name)

    text = ""
    for page in doc:
      text += page.get_text()

    cleaned_text = " ".join(text.split())

    if not cleaned_text.strip():
      return "Error: No text found, either it's not in pdf form or it's scanned image"

    return cleaned_text

  except Exception as e:
    return f"Error occurred: {str(e)}"

In [10]:
def extract_contact_info(text):
    """
    Find specific contact details using Regex patterns
    """

    email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
    emails = re.findall(email_pattern, text)

    # phone_pattern = r'^[6-9]\d{9}$'
    phone_pattern = r'[6-9]\d{9}'
    phones = re.findall(phone_pattern, text)

    linkedin = re.findall(r'linkedin\.com/in/[\w.-]+', text)

    github = re.findall(r'github\.com/[\w.-]+', text)

    return {
        "Emails": emails[0] if emails else "Email Not Found",
        "Phones": phones[0] if phones else "Phone Not Found",
        "LinkedIn": linkedin[0] if linkedin else "Link Not Found",
        "Github": github[0] if github else "Link Not Found"
    }

In [11]:
def resume_parser(pdf_file):
    raw_text = extract_text_from_pdf(pdf_file)

    contacts = extract_contact_info(raw_text)

    return contacts

In [12]:
interface = gr.Interface(
    fn = resume_parser,
    inputs = gr.File(label = "Upload Resume (PDF only)"),
    outputs = gr.JSON(label = "Structured Contact Info"),
    title = "AI Resume Parser"
)

In [ ]:
interface.launch(debug=True)